In [6]:
import numpy as np
from matplotlib import pyplot as plt
from math import sin, cos, sqrt, pi
from numpy import linalg

N = 10
x_start, x_end = 0.0, 1.0
y_start, y_end = 0.0, 0.0

xs = -1
xe = 2
ys = -2
ye = 2
step = 0.1
xn = int((xe - xs) / step) + 1
yn = int((ye - ys) / step) + 1

x = np.linspace(xs, xe, xn)
y = np.linspace(ys, ye, yn)
X, Y = np.meshgrid(x, y)

v = 1.0
alpha = 30.0*pi/180
dt = 1.0/N
t = 0.0
T = 3.0

x_vortex = [x_start + (x_end - x_start)/float(N)*(float(i) + 0.25) for i in range(N)]
y_vortex = [y_start + (y_end - y_start)/float(N)*(float(i) + 0.25) for i in range(N)]
x_control = [x_start + (x_end - x_start)/float(N)*(float(i) + 0.75) for i in range(N)]
y_control = [y_start + (y_end - y_start)/float(N)*(float(i) + 0.75) for i in range(N)]
x_vortex.append(x_end)
y_vortex.append(y_end)
A = ([[sqrt((x_control[j] - x_vortex[i])**2 + (y_control[j] - y_vortex[i])**2 ) for i in range(N+1)]  for j in range(N)])
A_coef = [[(x_control[i] - x_vortex[j])/abs(x_control[i] - x_vortex[j])/(2*pi*(A[i][j]))for j in range(N+1)] for i in range(N)]
b = [-v*sin(alpha)] * N
b.append(0.0)
A_coef.append([1.0]*(N+1))

def get_r(x_0, x, y_0, y):
    r = sqrt((x_0 - x)**2 + (y_0 - y)**2)
    return r

class free_vortex:
    def __init__(self, x, y, c):
        self.x = x
        self.y = y
        self.circulation = c


    def get_speed(self, x, y):
            r = sqrt((self.x - x)**2 + (self.y - y)**2)
            if r == 0:
                print("0")
                return [0, 0]  
            Us = self.circulation / (2 * pi * r)
            dx = Us * (y - self.y) / r
            dy = Us * (self.x - x) / r
            return [dx, dy]
    
end_vel_x = 0.0
end_vel_y = 0.0
free_vortex_Arr = []

while t < T:
    G = - np.linalg.solve(A_coef,b)
    for i in range(N):
        end_vel_x += G[i]*(y_end - y_vortex[i])/(2*pi*(x_vortex[i] - x_end)**2 + (y_vortex[i] - y_end)**2)
        end_vel_y += G[i]*(x_vortex[i] - x_end)/(2*pi*(x_vortex[i] - x_end)**2 + (y_vortex[i] - y_end)**2)
    for i, el in enumerate(free_vortex_Arr, 1):
        end_vel_x += el.circulation*(y_end - el.y)/(2*pi*(el.x - x_end)**2 + (el.y - y_end)**2)
        end_vel_y += el.circulation*(el.x - x_end)/(2*pi*(el.x - x_end)**2 + (el.y - y_end)**2)
    end_vel_x += v*cos(alpha)
    #res = ([[(np.sum([vortexes[k].get_speed(X[j][i], Y[j][i]) for k in range(N + 1)], axis=0)).tolist()\
    #         for i in range(xn)] for j in range(yn)])
    #U = [[el[0] + v * cos(alpha) for el in arr] for arr in res]
    #V = [[el[1] + v * sin(alpha) for el in arr] for arr in res]
    a = free_vortex(x = x_end + end_vel_x*dt , y = y_end + end_vel_y*dt, c = G[N])
    free_vortex_Arr.append(a)
    
    for i in range(len(free_vortex_Arr) - 1):
        for j in range(N + 1):
            free_vortex_Arr[i].x += G[j]*(free_vortex_Arr[i].y - y_vortex[j])\
            /(get_r(free_vortex_Arr[i].x, x_vortex[j], free_vortex_Arr[i].y, y_vortex[j]))*dt
            
            free_vortex_Arr[i].y += G[j]*(x_vortex[j] - free_vortex_Arr[i].x)\
            /(get_r(free_vortex_Arr[i].x, x_vortex[j], free_vortex_Arr[i].y, y_vortex[j]))*dt
        
        for j in range(len(free_vortex_Arr) - 1):
            if(i == j):
                break
            else: 
    
    #разобраться с дабавлением новых вихрей и перемещении старых
    
    for p in range(N):
        q = 0
        for e in free_vortex_Arr:
            b[p] += -e.circulation*(e.x - x_control[p])/(2*pi*((e.y - y_control[p])**2\
                                                                               + (e.x - x_control[p])**2 ))
            print(p," ",q ," ", b[p])
            q+=1
    
    t += dt


0   0   -0.4736272985813169
1   0   -0.4707438044266947
2   0   -0.4671539181974889
3   0   -0.46256268290510943
4   0   -0.45648513107329325
5   0   -0.44806496551016733
6   0   -0.4356390275615242
7   0   -0.4155143507181778
8   0   -0.3776758116421322
9   0   -0.2854124378729286
0   0   -0.443965058731933
0   1   -0.4258593311893076
1   0   -0.4373852857935029
1   1   -0.41749545746952094
2   0   -0.42904989865161425
2   1   -0.4069868103040645
3   0   -0.41814542261998155
3   1   -0.3933774088231626
4   0   -0.4032617806625833
4   1   -0.3750357168890153
5   0   -0.3817200803341969
5   1   -0.34891929459315973
6   0   -0.347725762426471
6   1   -0.30859325592165454
7   0   -0.2860191253668817
7   1   -0.23755938849806316
8   0   -0.14251430526850256
8   1   -0.07900355202354578
9   0   -0.3024766127098752
9   1   -0.21092883292516712
0   0   -0.3923003373461469
0   1   -0.37356332751393606
0   2   -0.3635738994820747
1   0   -0.37912982796730166
1   1   -0.35847581326864225
1   2  

1   8   0.07963101081353371
1   9   0.06269280342650776
2   0   0.145641459049922
2   1   0.16481902414421348
2   2   0.1746827220101445
2   3   0.18201623055237617
2   4   0.1863244166849988
2   5   0.1881128882016236
2   6   0.18152158268402843
2   7   0.1693179514018332
2   8   0.15358097742239424
2   9   0.13585854396299374
3   0   0.23994635800595546
3   1   0.2611367784813865
3   2   0.2717849238487617
3   3   0.27966328805928425
3   4   0.28427973001517887
3   5   0.2861924611251041
3   6   0.2792440808946128
3   7   0.2663956518861442
3   8   0.24985799099860712
3   9   0.23127525581869646
4   0   0.3689860012308147
4   1   0.39266013635928204
4   2   0.4042263870421875
4   3   0.41273580125922943
4   4   0.4177074676994808
4   5   0.41976287427532055
4   6   0.41241676476062167
4   7   0.39885168033235585
4   8   0.38142748819746153
4   9   0.361896753017317
5   0   0.5595671650978888
5   1   0.5863823476872978
5   2   0.5990370355211934
5   3   0.6082854980791259
5   4   0.61

4   2   0.3552646312847674
4   3   0.3626516436643571
4   4   0.367070921612564
4   5   0.36894616439584194
4   6   0.3619390586145178
4   7   0.34863692733457824
4   8   0.331197513524068
4   9   0.3113668973597611
4   10   0.29059108484211515
4   11   0.27000028275327753
4   12   0.25042861762389734
4   13   0.23246681037619196
5   0   0.5307910543916483
5   1   0.5533029036448377
5   2   0.5639259027194397
5   3   0.5718659459464853
5   4   0.5766100082540737
5   5   0.5786222299915476
5   6   0.5712104909723869
5   7   0.5571387461062741
5   8   0.5387105951899778
5   9   0.5177968242155937
5   10   0.495940399772696
5   11   0.474335774136295
5   12   0.45385295793624403
5   13   0.435098750650689
6   0   0.8749949217589359
6   1   0.9003295393783041
6   2   0.9118661896910847
6   3   0.9204474518856861
6   4   0.9255671976655584
6   5   0.9277378045790259
6   6   0.9198721842656391
6   7   0.9049364321602035
6   8   0.8854006921460256
6   9   0.8632787638485999
6   10   0.8402236

6   15   0.4889760302449151
6   16   0.4767026582701449
7   0   1.3443521595904944
7   1   1.371194111475156
7   2   1.382954554312147
7   3   1.3917118357865834
7   4   1.3969692557447126
7   5   1.3992186504244455
7   6   1.391047815571856
7   7   1.375311631109766
7   8   1.3545158039381817
7   9   1.3308170220543742
7   10   1.30605573779447
7   11   1.281660565372057
7   12   1.2586472631538015
7   13   1.2376940728107155
7   14   1.2192342351651213
7   15   1.2035326941039879
7   16   1.190739325671592
8   0   0.249710857447465
8   1   0.28065935794356633
8   2   0.2935461913574672
8   3   0.3030873418437917
8   4   0.30880876358998377
8   5   0.3112574858382438
8   6   0.3025324340273189
8   7   0.28570824582224447
8   8   0.2634909612795613
8   9   0.23822938553495554
8   10   0.21192018581522665
8   11   0.1860930669477684
8   12   0.1618146851282261
8   13   0.1397803615234077
8   14   0.12042195613257697
8   15   0.1039947605783515
8   16   0.0906362980684855
9   0   0.94744

8   8   -0.11525652551989157
8   9   -0.14054360011583011
8   10   -0.16689115553220352
8   11   -0.19275802677516898
8   12   -0.21707095000210466
8   13   -0.23913244684586094
8   14   -0.2585109574525059
8   15   -0.27495239732445964
8   16   -0.2883205540769335
8   17   -0.298561338891776
8   18   -0.30568406418918054
8   19   -0.30975420868288667
9   0   0.5773606039095547
9   1   0.6132101431227281
9   2   0.6272840148553092
9   3   0.6376495451321893
9   4   0.6438656584720561
9   5   0.64653144848233
9   6   0.6372037228044188
9   7   0.6191567188125818
9   8   0.595307841536635
9   9   0.5682340042991542
9   10   0.5401281132170145
9   11   0.5126476713670656
9   12   0.4869207972423644
9   13   0.46365989216322623
9   14   0.44329083150513104
9   15   0.42605363581210487
9   16   0.4120684060398814
9   17   0.4013737591336905
9   18   0.39394624245488774
9   19   0.3897072269283397
0   0   -0.7228180561818185
0   1   -0.7089535943829515
0   2   -0.7016785311814757
0   3   -0.

1   22   -1.0507996510312254
2   0   -0.8631538861465654
2   1   -0.8466971865944585
2   2   -0.8382939525592397
2   3   -0.8319269014041659
2   4   -0.8280986767278827
2   5   -0.8264677812494433
2   6   -0.8327542645611533
2   7   -0.8447235183384344
2   8   -0.8604739221784349
2   9   -0.8784577414826387
2   10   -0.8973781858203651
2   11   -0.9162076381721999
2   12   -0.9341755011176917
2   13   -0.9507255810700529
2   14   -0.965466669254038
2   15   -0.9781304451254598
2   16   -0.988540602896019
2   17   -0.9965923715122885
2   18   -1.0022403580985786
2   19   -1.0054926844282963
2   20   -1.0064095861450837
2   21   -1.0051046167227544
2   22   -1.0017463738890575
3   0   -0.7848206397846454
3   1   -0.7669026548303565
3   2   -0.7579348544673121
3   3   -0.7511598209455936
3   4   -0.7470896148368728
3   5   -0.7453559047224619
3   6   -0.7519665510229748
3   7   -0.7645554685960184
3   8   -0.7811079613557748
3   9   -0.7999782570814323
3   10   -0.8197913535680971
3   11 

6   4   -0.3380134476317464
6   5   -0.33577312558330685
6   6   -0.3438072506580282
6   7   -0.35889399931133453
6   8   -0.37841993318630074
6   9   -0.40032998270218334
6   10   -0.42300861592459443
6   11   -0.44526224972420236
6   12   -0.4662527205621812
6   13   -0.48540910733964776
6   14   -0.5023495876482235
6   15   -0.5168238749393292
6   16   -0.5286749258066525
6   17   -0.5378154748934117
6   18   -0.54421534200315
6   19   -0.5478965111811007
6   20   -0.5489336772282319
6   21   -0.5474580539135459
6   22   -0.5436618856148033
6   23   -0.5378007816937435
6   24   -0.5301914340299151
7   0   0.3207156547741546
7   1   0.35065423646075505
7   2   0.3638699507362901
7   3   0.37362287979926456
7   4   0.3794087201056846
7   5   0.3818467124753571
7   6   0.373277208792821
7   7   0.35719310879286664
7   8   0.33641922735506324
7   9   0.31317968237180466
7   10   0.28920821610076874
7   11   0.2657670978842067
7   12   0.2437266692826831
7   13   0.223667940138732
7   14

7   4   0.20663494448133227
7   5   0.20927215850462588
7   6   0.20034266561337563
7   7   0.18397113832410256
7   8   0.1632140153728919
7   9   0.14030346709056868
7   10   0.11687255188010559
7   11   0.09406586213206371
7   12   0.07266238281744139
7   13   0.05318561352763025
7   14   0.03598558683231913
7   15   0.02129403655273337
7   16   0.009259282912749599
7   17   -3.309861654107911e-05
7   18   -0.006549590522637443
7   19   -0.010305310132271615
7   20   -0.01136593243449283
7   21   -0.009853101549774961
7   22   -0.005950955627608478
7   23   8.924516083649511e-05
7   24   0.007949333720701782
7   25   0.017250634471803212
7   26   0.027564499264931874
8   0   -0.9349745207046082
8   1   -0.8940341076516832
8   2   -0.8771470835399126
8   3   -0.8649837873829838
8   4   -0.8579143976314436
8   5   -0.855000173850864
8   6   -0.8645945915392157
8   7   -0.8821468723272831
8   8   -0.9043199863607461
8   9   -0.9286880643876313
8   10   -0.9535011901834626
8   11   -0.97

6   1   -0.6516533497822503
6   2   -0.6350380829771133
6   3   -0.6232373394915461
6   4   -0.6165511764239339
6   5   -0.6138738067145297
6   6   -0.6226547404697711
6   7   -0.6383078462988644
6   8   -0.6578018509875463
6   9   -0.679097223466545
6   10   -0.7007746455395644
6   11   -0.7218547466158814
6   12   -0.7416634206320335
6   13   -0.7597342108644397
6   14   -0.7757423732062321
6   15   -0.7894622061461759
6   16   -0.8007402500305629
6   17   -0.8094792598356071
6   18   -0.8156298304977673
6   19   -0.8191878448016806
6   20   -0.8201965330644502
6   21   -0.8187519198227373
6   22   -0.8150098655819652
6   23   -0.8091921016305703
6   24   -0.8015883081856049
6   25   -0.7925523377697601
6   26   -0.7824937171236492
6   27   -0.771870562130556
6   28   -0.7611987452345942
7   0   0.050398319687331744
7   1   0.09377883096543421
7   2   0.11268704400852117
7   3   0.12593159800213302
7   4   0.1333785377930729
7   5   0.13634172896125454
7   6   0.12691837372971163
7  